In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import shfl
import torch
import copy
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv
from ClassifierModel import ClassifierModel

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
#"../data/COVIDGR1.0/centralized/cropped"
#partition_iid_3nodes_1.csv
args = {"data_path":"../data/COVIDGR1.0-Segmentadas", 
        "csv_path": "../partitions/partition_iid_3nodes_1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 1,
        "federated_rounds": 10,
        "epochs_per_FL_round": 5,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'

In [3]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0]), 
                'P' : lb1.transform(['P'])[0][0], 'N' : lb1.transform(['N'])[0][0]
              } 



In [4]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", folds=1, lambda_values = [0.05], batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

def classifier_builder( G_dict ):
    return ClassifierModel(G_dict, dict_labels, batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

def get_transformed_data(federated_data, cit_federated_government, lb1, lb2):
    t_federated_data = copy.deepcopy(federated_data)

    for i in range(federated_data.num_nodes()):
        data_node = federated_data[i]
        t_data_node = t_federated_data[i]
        data = data_node.query()._data
        labels = data_node.query()._label
        t_data, t_labels = cit_federated_government.global_model.transform_data(data, labels, lb1, lb2)
        t_data_node.query()._data = t_data
        t_data_node.query()._label = t_labels


    return t_federated_data

In [5]:
print("[INFO] Fetching federated data...")
federated_data, train_data, train_label, test_data, test_label, train_files, test_files = get_federated_data_csv(args["data_path"], args["csv_path"], lb1)
federated_data.configure_data_access(UnprotectedAccess())
print(len(train_data))
print(len(test_data))
print("[INFO] done")

[INFO] Fetching federated data...
681
171
[INFO] done


In [ ]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
cit_federated_government = shfl.federated_government.FederatedGovernment(cit_builder, federated_data, aggregator)
cit_federated_government.run_rounds(args["federated_rounds"], test_data, test_label)

Accuracy round 0
Training node 0
[INFO] weights = [1.         0.93965517]
[INFO] LAMBDA: 0.05


[Validating]: Acc_D: 0.5217: 100%|██████████| 23/23 [00:00<00:00, 27.24it/s]
[1/5] Loss_D: 0.3941 Acc_D: 0.5272 Loss_G_class1: 0.0241 Loss_G_class2: 0.0266: 100%|██████████| 202/202 [00:42<00:00,  4.75it/s]
  0%|          | 0/202 [00:00<?, ?it/s]


Valid Acc = 0.5652173913043478
Valid Loss = 0.6888010242710942


[2/5] Loss_D: 0.3826 Acc_D: 0.4851 Loss_G_class1: 0.0111 Loss_G_class2: 0.0111: 100%|██████████| 202/202 [00:42<00:00,  4.74it/s]
  0%|          | 0/202 [00:00<?, ?it/s]


Valid Acc = 0.5217391304347826
Valid Loss = 0.6899204798366713
EarlyStopping counter: 1 out of 10


...

[3/5] Loss_D: 0.3577 Acc_D: 0.5520 Loss_G_class1: 0.0099 Loss_G_class2: 0.0103: 100%|██████████| 202/202 [01:46<00:00,  1.90it/s]
  0%|          | 0/202 [00:00<?, ?it/s]


Valid Acc = 0.4782608695652174
Valid Loss = 0.7500765284766322
EarlyStopping counter: 1 out of 10


[4/5] Loss_D: 0.3592 Acc_D: 0.5371 Loss_G_class1: 0.0100 Loss_G_class2: 0.0094: 100%|██████████| 202/202 [01:47<00:00,  1.88it/s]
  0%|          | 0/202 [00:00<?, ?it/s]


Valid Acc = 0.4782608695652174
Valid Loss = 0.7572447437307109
EarlyStopping counter: 2 out of 10


[5/5] Loss_D: 0.3716 Acc_D: 0.4844 Loss_G_class1: 0.0099 Loss_G_class2: 0.0103:  16%|█▌        | 32/202 [00:17<01:29,  1.91it/s]

In [ ]:
metrics = cit_federated_government.global_model.evaluate(test_data, test_label)
print("CIT Classifier Results:")
print("Loss: {}".format(metrics[0]))
print("Acc: {}".format(metrics[1]))
print(metrics[2])

In [ ]:
t_federated_data = get_transformed_data(federated_data, cit_federated_government, lb1, lb2)

In [ ]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
G_dict = cit_federated_government.global_model._G_dict
x = lambda : classifier_builder(G_dict)
classifier_federated_government = shfl.federated_government.FederatedGovernment(x, t_federated_data, aggregator)
#classifier_federated_government = FederatedGovernment_2(classifier_builder(G_dict), t_federated_data, aggregator)
classifier_federated_government.run_rounds(args["federated_rounds"], test_data, test_label)

#print("[INFO] saving model ...")
#federated_government.global_model._model.save( os.path.join(args["output_path"], args["model_name"]) )
print("[INFO] done")

In [ ]:
metrics = classifier_federated_government.global_model.evaluate(test_data, test_label)
print("SDNET Classifier Results:")
print("Acc: {}".format(metrics[0]))
print("Acc_4: {}".format(metrics[1]))
print("No concuerda: {}".format(metrics[2]))
print(metrics[3])